**Monte Carlo Verification of Analytical BER Results**

This script verifies the analytical false positive, false negative,
and bit error rate (BER) expressions derived in the report using
Monte Carlo simulation. The simulation is used for verification
purposes only.



In [ ]:
import numpy as np
from math import erf, sqrt

# ---------- Helpers ----------
def Phi(x: float) -> float:
    # Standard normal CDF
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def Q(x: float) -> float:
    return 1.0 - Phi(x)

def ci95(p_hat: float, T: int):
    # 95% CI for Bernoulli proportion using normal approx
    se = np.sqrt(p_hat * (1 - p_hat) / T)
    return (p_hat - 1.96 * se, p_hat + 1.96 * se)

# ---------- Parameters (same as report) ----------
N = 100
M = 80
muX = 0.05
sigX2 = 0.01
sigX = np.sqrt(sigX2)
a, b = 0.0, 0.1
Vth = 1.5
Delta = 1.0

# ---------- Analytical ----------
muZ = N*muX - M*(a+b)/2
sigZ2 = N*sigX2 + M*((b-a)**2)/12
sigZ = np.sqrt(sigZ2)

PFP_a = Q((Vth - muZ) / sigZ)
PFN_a = Phi((Vth - (muZ + Delta)) / sigZ)
BER_a = 0.5*(PFP_a + PFN_a)

# ---------- Monte Carlo ----------
rng = np.random.default_rng(123)  # fixed seed -> repeatable
T = 1_000_000                      # increase for tighter match

SX = rng.normal(muX, sigX, size=(T, N)).sum(axis=1)
SY = rng.uniform(a, b, size=(T, M)).sum(axis=1)
Z = SX - SY

PFP_mc = float(np.mean(Z >= Vth))
PFN_mc = float(np.mean((Z + Delta) < Vth))
BER_mc = 0.5*(PFP_mc + PFN_mc)

# ---------- Print results as "table" ----------
print("\n=== Parameters ===")
print("muZ =", muZ)
print("sigZ^2 =", sigZ2)
print("sigZ =", sigZ)

print("\n=== Results Table ===")
print(f"{'Method':<20} {'P_FP':>12} {'P_FN':>12} {'BER':>12}")
print("-"*60)
print(f"{'Analytical':<20} {PFP_a:12.10f} {PFN_a:12.10f} {BER_a:12.10f}")
print(f"{'Monte Carlo':<20} {PFP_mc:12.10f} {PFN_mc:12.10f} {BER_mc:12.10f}")

print("\n=== 95% Confidence Intervals (MC) ===")
print("P_FP CI95:", ci95(PFP_mc, T))
print("P_FN CI95:", ci95(PFN_mc, T))



=== Parameters ===
muZ = 1.0
sigZ^2 = 1.0666666666666667
sigZ = 1.0327955589886444

=== Results Table ===
Method                       P_FP         P_FN          BER
------------------------------------------------------------
Analytical           0.3141493186 0.3141493186 0.3141493186
Monte Carlo          0.3143960000 0.3133520000 0.3138740000

=== 95% Confidence Intervals (MC) ===
P_FP CI95: (np.float64(0.3134860212542291), np.float64(0.31530597874577093))
P_FN CI95: (np.float64(0.31244284195402167), np.float64(0.3142611580459784))
